# Automated Design of Agents with DSPy

This notebook is inspired by the paper, ["Automated Design of Agentic Systems"](https://arxiv.org/abs/2408.08435) from [Shengrhan Hu](https://x.com/shengranhu), [Cong Lu](https://x.com/cong_ml), and [Jeff Clune](https://x.com/jeffclune). This notebook will illustrate how to construct a DSPy agent that designs DSPy agents inspired by the ideas in the paper.

Please note, this notebook only illustrates the "Generation" part of Meta Agent Search and is not a complete implementation of the paper. The paper further combines the "Generation" and "Search" aspects to create a system that can design agents based on the performance of previously explored agents.

Please let us know if you have any questions, feedback, or generally would like to see us continue to explore this idea.


In [4]:
import dspy

#llama3 = dspy.OllamaLocal(model="llama3.1")
gpt4 = dspy.OpenAI(model="gpt-4o", max_tokens=4000)

#print(llama3("say hello"))
print(gpt4("say hello"))

dspy.settings.configure(lm=gpt4)

['Hello! How can I assist you today?']


In [6]:
class AgentDesigner(dspy.Signature):
    """Your task is to design Agents for the application domain in the provided Python framework."""
    
    application_domain = dspy.InputField()
    agent_framework_description = dspy.InputField()
    #discovered_agent_archive = dspy.InputField() # uncomment this when using the "Search" part of Meta Agent Search
    optimal_agent = dspy.OutputField(desc="IMPORTANT!! ONLY OUTPUT THE CODE FOR THE AGENT IN THE PROVIDED PYTHON FRAMEWORK!!")

In [7]:
agent_designer = dspy.ChainOfThought(AgentDesigner)

In [8]:
dspy_framework_description="""
You will code the Agent with the DSPy Python Framework.

Here is an example of defining a program that retrieves information from a knowledge source and then uses the context to answer a question.

class Agent(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        context = self.retrieve(question).passages
        answer = self.generate_answer(context=context, question=question)
        return answer
        
Here is an example of a blog post writer Agent.

class Agent(dspy.Module):
    def __init__(self):
        self.question_to_blog_outline = dspy.ChainOfThought("question -> blog_outline")
        self.topic_to_paragraph = dspy.ChainOfThought("topic, contexts -> paragraph")
        self.proof_reader = dspy.ChainOfThought("blog_post -> proofread_blog_post")
        self.title_generator = dspy.ChainOfThought("blog_outline -> title")
    
    def forward(self, question):
        contexts = dspy.Retrieve(k=5)(question).passages
        contexts = "".join(contexts)
        raw_blog_outline = self.question_to_blog_outline(question=question, context=contexts).blog_outline
        blog_outline = raw_blog_outline.split(",") #Add type hint in expanded Signature
        blog = ""
        for topic in blog_outline:
            topic_contexts = dspy.Retrieve(k=5)(topic).passages
            topic_contexts = "".join(topic_contexts)
            blog += self.topic_to_paragraph(topic=topic, contexts=topic_contexts).paragraph
            blog += "\n \n"
        blog = self.proof_reader(blog_post=blog).proofread_blog_post
        title = self.title_generator(blog_outline=raw_blog_outline).title
        final_blog = f"{title} \n \n {blog}"
        return dspy.Prediction(blog=final_blog)

PLEASE NOTE!! It is extremely important that your Agent class is also named "Agent" as shown in the example!! THIS IS EXTREMELY IMPORTANT!!
"""

In [9]:
agent_design = agent_designer(
    application_domain="Produce Scientific Papers given a repository and a research direction as input.",
    agent_framework_description=dspy_framework_description
).optimal_agent

print(agent_design)

```python
import dspy

class Agent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.retrieve_info = dspy.Retrieve(k=10)
        self.generate_outline = dspy.ChainOfThought("research_direction, context -> paper_outline")
        self.expand_section = dspy.ChainOfThought("section, contexts -> paragraph")
        self.proof_reader = dspy.ChainOfThought("paper -> proofread_paper")
        self.title_generator = dspy.ChainOfThought("paper_outline -> title")

    def forward(self, repository, research_direction):
        # Retrieve relevant information from the repository
        context = self.retrieve_info(research_direction).passages
        context = "".join(context)
        
        # Generate an outline for the paper
        raw_paper_outline = self.generate_outline(research_direction=research_direction, context=context).paper_outline
        paper_outline = raw_paper_outline.split(",")
        
        # Expand each section of the outline into detailed par

In [10]:
gpt4.inspect_history(n=1)




Your task is to design Agents for the application domain in the provided Python framework.

---

Follow the following format.

Application Domain: ${application_domain}

Agent Framework Description: ${agent_framework_description}

Reasoning: Let's think step by step in order to ${produce the optimal_agent}. We ...

Optimal Agent: IMPORTANT!! ONLY OUTPUT THE CODE FOR THE AGENT IN THE PROVIDED PYTHON FRAMEWORK!!

---

Application Domain: Produce Scientific Papers given a repository and a research direction as input.

Agent Framework Description: You will code the Agent with the DSPy Python Framework. Here is an example of defining a program that retrieves information from a knowledge source and then uses the context to answer a question. class Agent(dspy.Module): def __init__(self, num_passages=3): super().__init__() self.retrieve = dspy.Retrieve(k=num_passages) self.generate_answer = dspy.ChainOfThought("context, question -> answer") def forward(self, question): context = self.retrie

'\n\n\nYour task is to design Agents for the application domain in the provided Python framework.\n\n---\n\nFollow the following format.\n\nApplication Domain: ${application_domain}\n\nAgent Framework Description: ${agent_framework_description}\n\nReasoning: Let\'s think step by step in order to ${produce the optimal_agent}. We ...\n\nOptimal Agent: IMPORTANT!! ONLY OUTPUT THE CODE FOR THE AGENT IN THE PROVIDED PYTHON FRAMEWORK!!\n\n---\n\nApplication Domain: Produce Scientific Papers given a repository and a research direction as input.\n\nAgent Framework Description: You will code the Agent with the DSPy Python Framework. Here is an example of defining a program that retrieves information from a knowledge source and then uses the context to answer a question. class Agent(dspy.Module): def __init__(self, num_passages=3): super().__init__() self.retrieve = dspy.Retrieve(k=num_passages) self.generate_answer = dspy.ChainOfThought("context, question -> answer") def forward(self, question)

In [11]:
agent_design = agent_designer(
    application_domain="Create a comprehensive travel itinerary given a destination and duration of stay.",
    agent_framework_description=dspy_framework_description
).optimal_agent

print(agent_design)

```python
import dspy

class Agent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.retrieve_info = dspy.Retrieve(k=5)
        self.generate_activities = dspy.ChainOfThought("destination, duration, context -> activities")
        self.organize_itinerary = dspy.ChainOfThought("activities, duration -> itinerary")
        self.proof_reader = dspy.ChainOfThought("itinerary -> proofread_itinerary")

    def forward(self, destination, duration):
        # Step 1: Retrieve information about the destination
        context = self.retrieve_info(destination).passages
        context = "".join(context)
        
        # Step 2: Generate a list of activities based on the destination and duration
        activities = self.generate_activities(destination=destination, duration=duration, context=context).activities
        
        # Step 3: Organize the activities into a coherent itinerary
        raw_itinerary = self.organize_itinerary(activities=activities, duration=du

In [12]:
gpt4.inspect_history(n=1)




Your task is to design Agents for the application domain in the provided Python framework.

---

Follow the following format.

Application Domain: ${application_domain}

Agent Framework Description: ${agent_framework_description}

Reasoning: Let's think step by step in order to ${produce the optimal_agent}. We ...

Optimal Agent: IMPORTANT!! ONLY OUTPUT THE CODE FOR THE AGENT IN THE PROVIDED PYTHON FRAMEWORK!!

---

Application Domain: Create a comprehensive travel itinerary given a destination and duration of stay.

Agent Framework Description: You will code the Agent with the DSPy Python Framework. Here is an example of defining a program that retrieves information from a knowledge source and then uses the context to answer a question. class Agent(dspy.Module): def __init__(self, num_passages=3): super().__init__() self.retrieve = dspy.Retrieve(k=num_passages) self.generate_answer = dspy.ChainOfThought("context, question -> answer") def forward(self, question): context = self.retr

'\n\n\nYour task is to design Agents for the application domain in the provided Python framework.\n\n---\n\nFollow the following format.\n\nApplication Domain: ${application_domain}\n\nAgent Framework Description: ${agent_framework_description}\n\nReasoning: Let\'s think step by step in order to ${produce the optimal_agent}. We ...\n\nOptimal Agent: IMPORTANT!! ONLY OUTPUT THE CODE FOR THE AGENT IN THE PROVIDED PYTHON FRAMEWORK!!\n\n---\n\nApplication Domain: Create a comprehensive travel itinerary given a destination and duration of stay.\n\nAgent Framework Description: You will code the Agent with the DSPy Python Framework. Here is an example of defining a program that retrieves information from a knowledge source and then uses the context to answer a question. class Agent(dspy.Module): def __init__(self, num_passages=3): super().__init__() self.retrieve = dspy.Retrieve(k=num_passages) self.generate_answer = dspy.ChainOfThought("context, question -> answer") def forward(self, questio

# Next Steps

Incorporating a `discovered_agent_archive` into the design process can revolutionize the creation of new agents. This repository of past designs and their resulting performance serves as a rich knowledge base. By drawing inspiration from successful agents, the meta agent designer can efficiently craft novel combinations and adaptations. This approach not only accelerates development but also fosters continuous improvement as the system learns from its growing collection of explored agent architectures.

If you found this notebook useful, please consider leaving a star on [Weaviate Recipes](https://github.com/weaviate/recipes).